In [1]:
from transformers import pipeline , AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn.functional as F
from utils import tokenize_hfmodel_inputs
device='cuda' if torch.cuda.is_available() else 'cpu'

/home/Testing_diff_models/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
low_compute=['mistralai/Mistral-7B-v0.3','google/gemma-2b','microsoft/phi-2','meta-llama/Llama-2-7b-hf','teknium/OpenHermes-2.5-Mistral-7B','meta-llama/Llama-3.1-8B','microsoft/Phi-3-mini-128k-instruct']
instruct_counter_part=['mistralai/Mistral-7B-Instruct-v0.3','meta-llama/Llama-2-7b-chat-hf']

Text completions

In [3]:
model_name=low_compute[3]

In [4]:
model=AutoModelForCausalLM.from_pretrained(model_name).to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:37<00:00, 18.50s/it]


In [5]:
sample_text="How to make a sandwich"


tokens,tokeniser=tokenize_hfmodel_inputs(sample_text,model_name,'cpu')
x=tokens.to(device)
for _ in range(50):
    ops=model(input_ids=x)
    logits=ops.logits
    probs=F.softmax(logits[:,-1,:],dim=-1)

    next_token=torch.multinomial(probs.view(probs.shape[1]),1)

    x=torch.cat([x,next_token.view(1,1)],dim=-1)
op=''
for i in x.tolist():
    op=op+tokeniser.decode(i)

/home/Testing_diff_models/utils.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens=torch.tensor(tokens).view(1,tokens.shape[0])


In [6]:
print(op)

<s> How to make a sandwich- Smashburger style
Words by Roberto Friedman, Smashburger
Wing, fries, cheese curds…you make the call.
For the Taskmasters, Royal Payne, Sophia Hilton and K


Chat

In [2]:
from transformers import pipeline , AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList
import torch
import torch.nn.functional as F
from utils import tokenize_hfmodel_inputs
device='cuda' if torch.cuda.is_available() else 'cpu'

/home/Testing_diff_models/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name=instruct_counter_part[1]

In [4]:
model=AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokeniser=AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [01:28<00:00, 44.01s/it]


In [5]:
class StopOnEos(StoppingCriteria):
    def __call__(self, input_ids, scores, **kwargs):
        # Check if the last generated token is the EOS token
        eos_token_id = 50256  # Change this based on your model's tokenizer
        return input_ids[0, -1] == eos_token_id

stopping_criteria = StoppingCriteriaList([StopOnEos()])

In [6]:
sample_text="How to make a sandwich"
messages = [{"role": "user", "content": sample_text}]
prompt = tokeniser.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
input_ids = tokeniser(prompt, return_tensors="pt").to(device)
output_ids = model.generate(input_ids.input_ids,max_length=500,do_sample=True,temperature=0.7,top_p=0.9)
response = tokeniser.decode(output_ids[0], skip_special_tokens=True)
print(response)

[INST] How to make a sandwich [/INST]  Making a sandwich is a simple process that requires a few basic ingredients and some simple steps. Here's a step-by-step guide on how to make a sandwich:

Ingredients:

* Bread (white, whole wheat, or your choice of bread)
* Filling ingredients (e.g., meats, cheeses, vegetables, condiments)
* Spread (e.g., mayonnaise, mustard, hummus)

Instructions:

1. Lay out the bread slices: Place two slices of bread on a clean work surface.
2. Add the filling ingredients: Place the filling ingredients on one of the bread slices, starting from the bottom. You can use meats like turkey, ham, or roast beef, cheeses like cheddar, Swiss, or mozzarella, vegetables like lettuce, tomato, or avocado, and condiments like mayonnaise, mustard, or hummus.
3. Add the spread: Spread a layer of spread on top of the filling ingredients.
4. Add the other bread slice: Place the other bread slice on top of the filling, spread side down.
5. Cut the sandwich: Use a sharp knife to 

In [8]:
messages.append({"role": "assistant", "content": response})
messages.append({"role": "user", "content": "Given context Give me a crisp 2 line responce of it"})

In [9]:
messages

[{'role': 'user', 'content': 'How to make a sandwich'},
 {'role': 'assistant',
  'content': "[INST] How to make a sandwich [/INST]  Making a sandwich is a simple process that requires a few basic ingredients and some simple steps. Here's a step-by-step guide on how to make a sandwich:\n\nIngredients:\n\n* Bread (white, whole wheat, or your choice of bread)\n* Filling ingredients (e.g., meats, cheeses, vegetables, condiments)\n* Spread (e.g., mayonnaise, mustard, hummus)\n\nInstructions:\n\n1. Lay out the bread slices: Place two slices of bread on a clean work surface.\n2. Add the filling ingredients: Place the filling ingredients on one of the bread slices, starting from the bottom. You can use meats like turkey, ham, or roast beef, cheeses like cheddar, Swiss, or mozzarella, vegetables like lettuce, tomato, or avocado, and condiments like mayonnaise, mustard, or hummus.\n3. Add the spread: Spread a layer of spread on top of the filling ingredients.\n4. Add the other bread slice: Place

In [10]:
prompt = tokeniser.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
input_ids = tokeniser(prompt, return_tensors="pt").to(device)

output_ids = model.generate(input_ids.input_ids, max_length=3400, do_sample=True, top_p=0.9)
response = tokeniser.decode(output_ids[0], skip_special_tokens=True)

print(response)


[INST] How to make a sandwich [/INST] [INST] How to make a sandwich [/INST]  Making a sandwich is a simple process that requires a few basic ingredients and some simple steps. Here's a step-by-step guide on how to make a sandwich:

Ingredients:

* Bread (white, whole wheat, or your choice of bread)
* Filling ingredients (e.g., meats, cheeses, vegetables, condiments)
* Spread (e.g., mayonnaise, mustard, hummus)

Instructions:

1. Lay out the bread slices: Place two slices of bread on a clean work surface.
2. Add the filling ingredients: Place the filling ingredients on one of the bread slices, starting from the bottom. You can use meats like turkey, ham, or roast beef, cheeses like cheddar, Swiss, or mozzarella, vegetables like lettuce, tomato, or avocado, and condiments like mayonnaise, mustard, or hummus.
3. Add the spread: Spread a layer of spread on top of the filling ingredients.
4. Add the other bread slice: Place the other bread slice on top of the filling, spread side down.
5. C

Code

In [ ]:
model_name="google/gemma-2b-it"

In [ ]:
sample_text="write a c program to print first 5 numbers of fibonacci series."
model=AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokeniser=AutoTokenizer.from_pretrained(model_name)
messages = [{"role": "user", "content": sample_text}]
prompt = tokeniser.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
input_ids = tokeniser(prompt, return_tensors="pt").to(device)
output_ids = model.generate(input_ids.input_ids,max_length=200,do_sample=True,temperature=0.7,top_p=0.9)
response = tokeniser.decode(output_ids[0], skip_special_tokens=True)
print(response)